In [18]:
%pip install -r requirements.txt

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached pyarrow-20.0.0-cp312-cp312-macosx_12_0_arm64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.3.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached xxhash-3.5.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py312-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached aiohttp-3.12.13-cp312-cp312-macosx_11_0_arm64.whl.metadata (7.6 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.7.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (18 kB)
  Using cached propcache-0.3.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp312-cp312-macosx

In [1]:
from transformers import CLIPModel, CLIPTokenizer, CLIPImageProcessor, AutoModelForCausalLM, AutoTokenizer
from PIL import Image
import torch
import torch.nn as nn

clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
vision_encoder = clip_model.vision_model
text_encoder = clip_model.text_model
image_processor = CLIPImageProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-base-patch32")

qwen_model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B-Base")
qwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B-Base")


/Users/rosh/MLX-Week4-CLIP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
items = ["cat", "dog", "horse", "bird", "car", "person", "tree", "house", "book", "phone"]
classifiers = [f"a photo of a {item}" for item in items]

text_inputs = tokenizer(classifiers, return_tensors="pt")
text_features = text_encoder(**text_inputs).pooler_output
text_features_proj = clip_model.text_projection(text_features)

for item, classifier in zip(items, classifiers):
    image = Image.open(f"images/{item}.jpg")
    image_inputs = image_processor(image, return_tensors="pt")
    image_features = vision_encoder(**image_inputs).pooler_output
    image_features_proj = clip_model.visual_projection(image_features)
    
    # Compute similarity to all classifier texts
    similarities = (image_features_proj @ text_features_proj.T).squeeze(0)
    best_idx = similarities.argmax().item()
    best_label = classifiers[best_idx]
    best_score = similarities[best_idx].item()
    
    print(f"{item}: best match is '{best_label}' (score: {best_score:.3f})")

cat: best match is 'a photo of a cat' (score: 30.380)
dog: best match is 'a photo of a dog' (score: 35.998)
horse: best match is 'a photo of a horse' (score: 36.964)
bird: best match is 'a photo of a bird' (score: 30.397)
car: best match is 'a photo of a car' (score: 23.982)
person: best match is 'a photo of a person' (score: 27.901)
tree: best match is 'a photo of a tree' (score: 30.656)
house: best match is 'a photo of a house' (score: 29.412)
book: best match is 'a photo of a book' (score: 40.067)
phone: best match is 'a photo of a phone' (score: 32.127)


In [3]:
clip_dim = clip_model.visual_projection.out_features  # 512
qwen_dim = qwen_model.model.embed_tokens.embedding_dim  # 4096

adapter = nn.Sequential(
    nn.Linear(clip_dim, qwen_dim),
    nn.LayerNorm(qwen_dim),
    nn.GELU(),
    nn.Linear(qwen_dim, qwen_dim),
)

for item in items:
    image = Image.open(f"images/{item}.jpg")
    image_inputs = image_processor(image, return_tensors="pt")
    image_features = vision_encoder(**image_inputs).pooler_output
    image_features_proj = clip_model.visual_projection(image_features)
    image_latent = adapter(image_features_proj)
    attention_mask = torch.ones(1, 1)
    position_ids = torch.zeros(1, 1)

    generated_ids = qwen_model.generate(inputs_embeds=image_latent.unsqueeze(1), attention_mask=attention_mask, position_ids=position_ids, max_new_tokens=30, do_sample=True, temperature=0.8, pad_token_id=qwen_tokenizer.pad_token_id, eos_token_id=qwen_tokenizer.eos_token_id)

    print(f"{item}: {qwen_tokenizer.decode(generated_ids[0])}")

cat: zzzoj CONTzอนzojz here IMPeezz CONTzzózez zzzzezzz
dog: hanhan ln xyz Sudokuhanhan lnhanhan xyz ^ _ aa Sudokuhan Sudoku bb ^ aa ij vnhan bvhanhan ee llna oo


KeyboardInterrupt: 

In [27]:
from datasets import load_dataset
import os

# Load Flickr30k dataset from Hugging Face
print("Loading Flickr30k dataset...")
dataset = load_dataset("nlphuji/flickr30k")


print(f"Dataset loaded: {dataset}")
print(f"Available splits: {list(dataset.keys())}")
print(f"Test set: {len(dataset['test'])} samples")

# Convert to the format your training code expects
captions = {}
for item in dataset['test']:  # Changed from 'train' to 'test'
    img_filename = item['filename']  # Changed from 'image_file_name' to 'filename'
    caption = item['caption']  # Changed from 'sentence' to 'caption'
    captions[img_filename] = caption  # Don't use setdefault with append
    
print(f"Loaded {len(captions)} images with captions")



Loading Flickr30k dataset...
Dataset loaded: DatasetDict({
    test: Dataset({
        features: ['image', 'caption', 'sentids', 'split', 'img_id', 'filename'],
        num_rows: 31014
    })
})
Available splits: ['test']
Test set: 31014 samples
Loaded 31014 images with captions


In [ ]:
from PIL import Image
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim


device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

clip_model = clip_model.to(device)
clip_model = clip_model.to(device)
vision_encoder = vision_encoder.to(device)
qwen_model = qwen_model.to(device)
adapter = adapter.to(device)


# dont change qwen weights
for param in qwen_model.parameters():
    param.requires_grad = False

optimizer = optim.AdamW(adapter.parameters(), lr=1e-5)


sample_size = 10  # Instead of 31,014 samples
dataset = dataset['test'].select(range(sample_size))

image_files = list(captions.keys())  # Use all images for demo
for epoch in range(3):
    for item in tqdm(dataset):
        # Get image and caption
        image = item['image']
        caption = item['caption'][0]
        
        # Process image
        image_inputs = image_processor(image, return_tensors="pt").to(device)
        image_features = vision_encoder(**image_inputs).pooler_output
        image_features_proj = clip_model.visual_projection(image_features)
        image_latent = adapter(image_features_proj)

        # Tokenize the caption
        input_ids = qwen_tokenizer(caption, return_tensors="pt", truncation=True, max_length=32).input_ids.to(device)
        
        # Create the full sequence: [image_embedding] + [caption_tokens]
        image_latent_seq = image_latent.unsqueeze(1)  # [1, 1, hidden_dim]
        
        # Get text embeddings for the caption
        text_embeds = qwen_model.model.embed_tokens(input_ids)
        
        # Concatenate: image embedding + text embeddings
        full_embeddings = torch.cat([image_latent_seq, text_embeds], dim=1)
        
        # Create labels: -100 for image position, actual tokens for text
        batch_size, seq_len = full_embeddings.size(0), full_embeddings.size(1)
        labels = torch.full((batch_size, seq_len), -100, dtype=torch.long, device=device)
        labels[:, 1:1+input_ids.size(1)] = input_ids  # Fill text positions with actual tokens
        
        # Forward pass
        outputs = qwen_model(inputs_embeds=full_embeddings, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item():.4f}")

print("Training loop complete (demo).")

# Save the trained models
print("Saving trained models...")
torch.save(adapter.state_dict(), "adapter.pth")
print("Models saved successfully!")

Using device: mps


  1%|          | 1/100 [00:15<24:57, 15.13s/it]

Loss: 3.9998


  2%|▏         | 2/100 [00:21<16:22, 10.03s/it]

Loss: 4.5520


...

Loss: 4.2975


 32%|███▏      | 32/100 [06:21<23:27, 20.71s/it]

Loss: 4.6432


 33%|███▎      | 33/100 [06:28<18:38, 16.70s/it]

Loss: 5.1250


In [14]:
import os

device = "cuda" if torch.cuda.is_available() else "cpu"



adapter.load_state_dict(torch.load("adapter.pth"))
adapter = adapter.float()  # Ensure adapter is in full precision
qwen_model = qwen_model.float() 


image_folder = "images"
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg")]

for img_file in image_files:
    image = Image.open(os.path.join(image_folder, img_file)).convert("RGB")
    image_inputs = image_processor(image, return_tensors="pt").to(device)
    image_features = vision_encoder(**image_inputs).pooler_output
    image_features_proj = clip_model.visual_projection(image_features)
    image_latent = adapter(image_features_proj)

    # Generate text from image latent
    attention_mask = torch.ones(1, 1, dtype=torch.long).to(device)
    position_ids = torch.zeros(1, 1, dtype=torch.long).to(device)
    generated_ids = qwen_model.generate(
        inputs_embeds=image_latent.unsqueeze(1),
        attention_mask=attention_mask,
        position_ids=position_ids,
        max_new_tokens=30,
        do_sample=True,
        temperature=0.8,
        pad_token_id=qwen_tokenizer.pad_token_id,
        eos_token_id=qwen_tokenizer.eos_token_id,
    )
    description = qwen_tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(f"{img_file}: {description}")

dog.jpg:  Seg_Pos中着中的_PosEscPos中Esc*/着》着著乘著着否定中中的著書着着ÆFLSeg SegPOS
horse.jpg: )ibilitiesusherDrivenusherushingollywood Kempusherusherushingstanding Kempstandingereusher Possusherusherusherusher tậnusher acceleratedusher existsushing Possereeusher
book.jpg: 1  ( ',",


1的 
9  0a x T9

 N",  T 3 2

  
bird.jpg: Asked voting voting inter pageNumber hoe veryShareAskedAskedette veryboard votingAskedAskedAsked very very pageNumberPubMed voting votingAsked very very pager hoeAskedboard
person.jpg: ловлов tears?








ловлов



 значительноловлов значительноLOT значительнолов tearsловLOTлов.



лов значительно.


ловловлов tearsловлов.



house.jpg: ??^^^^%%uu^''-
eeoooooooollaaee,,ee,,oooooooo!ffffcccc^^^^aaaaaaaa!!','aaaaooooooorrffffff                                                                                                                                
cat.jpg: .Invoke揮.Invoke MonoBehaviour.Invoke.Invoke_VOID.Invokeobre#pragma.Invoke#pragma.Invoke.Invokeobreobre天空.Invoke slagDetr